In [77]:
# TESTING ZONE

import pybaseball as pb
import pandas as pd
from datetime import datetime
from os import path

dir = r'C:\Users\dbann\Documents\Baseball\GSQ\data'

gl = pd.read_csv(
    path.join(dir, 'pgamelog24.csv'))

# # Define the start and end dates
# start_date = '2024-03-20'
# end_date = '2024-08-19'
# dates = pd.date_range(start=start_date, end=end_date)

gl['Date'].max()
datetime.today().strftime('%Y-%m-%d')


'2024-08-23'

In [67]:
# # SKIP THIS CODE
# # 
# # original get data for march to august 19. Only run once, run the following cells to update the data, now saved locally.

# import pybaseball as pb
# import pandas as pd

# # Create the empty dataframe with the specified columns
# pgamelog24 = pd.DataFrame(columns=['Date','PitcherID','Pitcher','G','GS','IP','H','K','BB','R','HR'])

# # Define the start and end dates
# start_date = '2024-03-20'
# end_date = '2024-08-19'
# dates = pd.date_range(start=start_date, end=end_date)

# # Loop through each date
# for date in dates:
#     start_dt = date.strftime('%Y-%m-%d')
    
#     try:
#         # Fetch pitching stats for that specific date
#         daily_gl = pb.pitching_stats_range(start_dt=start_dt, end_dt=start_dt)
        
#         # Add the Date column
#         daily_gl['Date'] = start_dt
        
#         # Select and rename the relevant columns
#         daily_gl = daily_gl[['Date', 'mlbID', 'Name', 'G', 'GS', 'IP', 'H', 'SO', 'BB', 'ER', 'HR']]
#         daily_gl.columns = ['Date', 'PitcherID', 'Pitcher', 'G', 'GS', 'IP', 'H', 'K', 'BB', 'R', 'HR']
        
#         # Append to the pgamelog24 dataframe
#         pgamelog24 = pd.concat([pgamelog24, daily_gl], ignore_index=True)
        
#     except Exception as e:
#         # Skip this date if an error occurs (e.g., no games or data issues)
#         pass

# # Display the populated dataframe
# pgamelog24

# pgamelog24.to_csv(r'C:\Users\dbann\Documents\Baseball\GSQ\data\pgamelog24.csv')


C:\Users\dbann\AppData\Local\Temp\ipykernel_17160\760713553.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pgamelog24 = pd.concat([pgamelog24, daily_gl], ignore_index=True)


In [ ]:
# updating it using max date in file and today's date.

import pybaseball as pb
import pandas as pd
from datetime import datetime
from os import path

dir = r'C:\Users\dbann\Documents\Baseball\GSQ\data'

gl = pd.read_csv(
    path.join(dir, 'pgamelog24.csv'))

gl.drop(columns=['Unnamed: 0'])

# Define the start and end dates
start_date = gl['Date'].max()
end_date = datetime.today().strftime('%Y-%m-%d')
dates = pd.date_range(start=start_date, end=end_date)

# Loop through each date
for date in dates:
    start_dt = date.strftime('%Y-%m-%d')
    
    try:
        # Fetch pitching stats for that specific date
        daily_gl = pb.pitching_stats_range(start_dt=start_dt, end_dt=start_dt)
        
        # Add the Date column
        daily_gl['Date'] = start_dt
        
        # Select and rename the relevant columns
        daily_gl = daily_gl[['Date', 'mlbID', 'Name', 'G', 'GS', 'IP', 'H', 'SO', 'BB', 'ER', 'HR']]
        daily_gl.columns = ['Date', 'PitcherID', 'Pitcher', 'G', 'GS', 'IP', 'H', 'K', 'BB', 'R', 'HR']
        
        # Append to the pgamelog24 dataframe
        gl = pd.concat([gl, daily_gl], ignore_index=True)
        
    except Exception as e:
        # Skip this date if an error occurs (e.g., no games or data issues)
        pass

# Display the populated dataframe

gl.to_csv(r'C:\Users\dbann\Documents\Baseball\GSQ\data\pgamelog24.csv')

In [50]:
# Putting together the actual game log portion of the dataset, which calcs Game Score 2
import pybaseball as pb
import pandas as pd
from os import path

dir = r'C:\Users\dbann\Documents\Baseball\GSQ\data'

gl = pd.read_csv(
    path.join(dir, 'pgamelog24.csv'))

spgl24 = pd.DataFrame(gl[gl['GS'] == 1])
spgl24['PitcherID'] = spgl24['PitcherID'].astype(str)

# Convert IP to Outs

spgl24['Outs'] = 10*spgl24['IP'] - 7*round(spgl24['IP'],0)
spgl24['Outs'].sample(20)

spgl24['GS2'] = (40 + 2*spgl24['Outs'] + spgl24['K'] - 2*spgl24['BB'] - 2*spgl24['H'] - 3*spgl24['R'] - 6*spgl24['HR'])

In [47]:
# generate summary table

gsq = pd.DataFrame(spgl24.groupby('PitcherID',as_index=False).agg(
    Pitcher = ('Pitcher','max'),
    GS = ('GS','sum'),
    # IP = ('IP', 'sum'),
    K = ('K', 'sum'),
    BB = ('BB', 'sum'),
    GS2 = ('GS2', 'mean'),
    GSD = ('GS2', 'std')
))

gsq['GSD'] = gsq['GSD'].fillna(value=1).replace(0,1)

gsq['GSQ'] = gsq['GS2'] / gsq['GSD']

gsq[['PitcherID','Pitcher','GSQ','GS2','GSD','GS','K']].query('GS >= 10').sort_values('GSQ',ascending=False).head(10)

,PitcherID,Pitcher,GSQ,GS2,GSD,GS,K
42,592866,Trevor Williams,7.642656,59.181818,7.743619,11,47
175,669203,Corbin Burnes,7.473885,61.684211,8.253300,19,110
84,625643,Reynaldo L\xc3\xb3pez,5.523902,61.375000,11.110806,16,93
236,680730,Mitchell Parker,5.035699,55.437500,11.008898,16,74
45,594902,Ben Lively,5.020790,55.000000,10.954451,15,72
122,657376,Clarke Schmidt,4.683693,58.727273,12.538667,11,67
282,694973,Paul Skenes,4.681316,63.000000,13.457753,10,78
217,676710,Kutter Crawford,4.636916,56.947368,12.281303,19,108
154,666157,Nick Lodolo,4.520939,59.333333,13.124116,12,73
64,607625,Seth Lugo,4.399647,63.052632,14.331293,19,110


In [48]:
# Start running some diagnostics!
gsq.query('GS >= 10').sort_values('GSQ',ascending=False).describe()

spgl24.query('PitcherID == "592866"').sort_values('GS2',ascending=False)

,Unnamed: 0,Date,PitcherID,Pitcher,G,GS,IP,H,K,BB,R,HR,Outs,GS2
4572,4572,5/7/2024,592866,Trevor Williams,1,1,5.0,2,8,0,0,0,15.0,74.0
2699,2699,4/20/2024,592866,Trevor Williams,1,1,6.0,3,4,0,1,0,18.0,71.0
7080,7080,5/30/2024,592866,Trevor Williams,1,1,5.2,4,4,2,1,0,17.0,63.0
833,833,4/3/2024,592866,Trevor Williams,1,1,5.1,3,5,2,2,0,16.0,61.0
6548,6548,5/25/2024,592866,Trevor Williams,1,1,5.0,5,8,0,1,1,15.0,59.0
1356,1356,4/8/2024,592866,Trevor Williams,1,1,5.0,3,2,3,1,0,15.0,57.0
2025,2025,4/14/2024,592866,Trevor Williams,1,1,5.1,6,7,1,3,0,16.0,56.0
3946,3946,5/1/2024,592866,Trevor Williams,1,1,5.0,5,3,4,0,0,15.0,55.0
3343,3343,4/26/2024,592866,Trevor Williams,1,1,5.0,5,1,2,1,0,15.0,54.0
5364,5364,5/14/2024,592866,Trevor Williams,1,1,5.0,4,2,1,1,1,15.0,53.0


In [ ]:
#Streamlit App

import streamlit as st

st.title('MLB Pitching Consistency Leaderboard')